In [ ]:
import pandas as pd
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
data2 = pd.read_pickle('z_data_new2/total_raw/data_raw_20241223_py.pkl')
# data3 = pd.read_pickle('z_data/data_with_growth/data_1230_cleaned_merged.pkl')

In [ ]:
print(data2.columns)

In [ ]:
# data2['feature_split'] = data2['feature'].str.split(';')

# # 过滤掉 None 值
# data2 = data2.dropna(subset=['feature_split'])
# max_splits_per_row = data2['feature_split'].apply(len).max()
# print(f'每行最多有 {max_splits_per_row} 个拆分数据')

In [ ]:

# data2['feature_split'] = data2['feature'].str.split(';')
# data2 = data2.dropna(subset=['feature_split'])
# data2['split_count'] = data2['feature_split'].apply(len)
# most_common_count = data2['split_count'].value_counts().idxmax()
# most_common_frequency = data2['split_count'].value_counts().max()
# print(f'频数最高的拆分数据个数是 {most_common_count}，频数为 {most_common_frequency}')

# plt.figure(figsize=(10, 6))
# sns.histplot(data2['split_count'], bins=range(1, 20), kde=False)
# plt.axvline(most_common_count, color='r', linestyle='--', label=f'频数最高: {most_common_count}')
# plt.xlabel('每行数据个数')
# plt.ylabel('频数')
# plt.title('每行数据个数的分布图')
# plt.legend()
# plt.show()

In [ ]:
def process_and_output(data):
    data['feature_split'] = data['feature'].str.split(';')
    data['split_count'] = data['feature_split'].apply(lambda x: len(x) if x is not None else 0)
    
    all_features = [feature for sublist in data['feature_split'].dropna() for feature in sublist]
    feature_counts = Counter(all_features)

    def process_features(features, feature_counts, max_len=6):
        if features is None:
            features = []
        if len(features) > max_len:
            features = sorted(features, key=lambda x: feature_counts[x], reverse=True)[:max_len]
        # 如果不足 max_len 个，填充 -1
        features.extend([-1] * (max_len - len(features)))
        return features

    data[['feature1', 'feature2', 'feature3', 'feature4', 'feature5', 'feature6']] = data['feature_split'].apply(lambda x: pd.Series(process_features(x, feature_counts)))
    data = data.drop(columns=['feature_split', 'split_count', 'feature'])
    
    return data

df = process_and_output(data2)
print(df.columns)

In [ ]:


# 获取频数最高的前20个值
top_brand_real_counts = brand_real_counts.nlargest(20)
top_audience_counts = audience_counts.nlargest(20)
top_scenario_counts = scenario_counts.nlargest(20)

# 绘制直方图
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
sns.barplot(x=top_brand_real_counts.values, y=top_brand_real_counts.index)
plt.title('Brand Real Distribution')
plt.xlabel('Frequency')
plt.ylabel('Brand Real')

plt.subplot(1, 3, 2)
sns.barplot(x=top_audience_counts.values, y=top_audience_counts.index)
plt.title('Audience Distribution')
plt.xlabel('Frequency')
plt.ylabel('Audience')

plt.subplot(1, 3, 3)
sns.barplot(x=top_scenario_counts.values, y=top_scenario_counts.index)
plt.title('Scenario Distribution')
plt.xlabel('Frequency')
plt.ylabel('Scenario')

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# 定义函数拆分列并计算拆分后的个数
def split_and_count(data, column):
    data[f'{column}_split'] = data[column].str.split(';')
    data = data.dropna(subset=[f'{column}_split'])
    data[f'{column}_count'] = data[f'{column}_split'].apply(len)
    return data

# 对 scenario、brand_real、audience 列进行拆分并计算个数
columns_to_split = ['scenario', 'brand_real', 'audience']
for column in columns_to_split:
    data2 = split_and_count(data2, column)

# 绘制频数直方图
plt.figure(figsize=(18, 6))

for i, column in enumerate(columns_to_split, 1):
    plt.subplot(1, 3, i)
    sns.histplot(data2[f'{column}_count'], bins=range(1, data2[f'{column}_count'].max() + 1), kde=False)
    plt.xlabel(f'{column} 拆分后个数')
    plt.ylabel('频数')
    plt.title(f'{column} 拆分后个数的分布图')

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

# 假设 data2 已经定义并包含数据

# 定义一个函数来处理拆分列并保留前2个
def split_and_expand(data, column, max_len=2):
    data[f'{column}_split'] = data[column].str.split(';')
    
    all_items = [item for sublist in data[f'{column}_split'].dropna() for item in sublist]
    item_counts = Counter(all_items)
    
    def process_items(items, item_counts, max_len):
        if items is None:
            items = []
        if len(items) > max_len:
            items = sorted(items, key=lambda x: item_counts[x], reverse=True)[:max_len]
        # 如果不足 max_len 个，填充 -1
        items.extend([-1] * (max_len - len(items)))
        return items
    
    # 应用函数到每行
    expanded_columns = data[f'{column}_split'].apply(lambda x: pd.Series(process_items(x, item_counts, max_len)))
    expanded_columns.columns = [f'{column}_{i+1}' for i in range(max_len)]
    
    data = pd.concat([data, expanded_columns], axis=1)
    data.drop(columns=[f'{column}_split'], inplace=True)
    data.drop(columns=[column], inplace=True)
    
    return data

columns_to_process = ['scenario', 'brand_real', 'audience']
for column in columns_to_process:
    data2 = split_and_expand(data2, column)

print(data2.head())

In [ ]:
print(data2.columns)